In [ ]:
"""Наши данные представляют собой четыре таблицы:

ratings1 и ratings2 — таблицы с данными о выставленных пользователями оценках фильмов. 
Они имеют одинаковую структуру и типы данных — на самом деле это две части одной таблицы 
с оценками фильмов.
userId — уникальный идентификатор пользователя, который выставил оценку;
movieId — уникальный идентификатор фильма;
rating — рейтинг фильма.

dates — таблица с датами выставления всех оценок.
date — дата и время выставления оценки фильму.

movies — таблица с информацией о фильмах.
movieId — уникальный идентификатор фильма;
title — название фильма и год его выхода;
genres — жанры фильма."""
import pandas as pd
movies_df = pd.read_csv('data_test/movies.csv')
#Сколько уникальных фильмов представлено в таблице movies?
movies_df['movieId'].nunique()
#Сколько уникальных пользователей в таблице ratings1?
ratings1_df = pd.read_csv('data_test/ratings1.csv')
ratings1_df['userId'].nunique()
#В каком году было выставлено больше всего оценок?...используйте таблицу dates.
dates_df = pd.read_csv('data_test/dates.csv')
dates_df = pd.to_datetime(dates_df,yearfirst=True)


ValueError: to assemble mappings requires at least that [year, month, day] be specified: [day,month,year] is missing